In [ ]:
"""
For ChIP-seq data, download ENCODE metadata TSV file and filter it
"""
# user‐configurable parameters
encode_tsv = "/mnt/disk/home/akang/projects/manuscript/data/input/TF_files.tsv"
assay_type  = "TF"          # "TF" or "histone"
min_peaks   = 1000
n_threads   = 4
output_dir  = "../data/bed"

In [ ]:
# imports and functions
